# Learning Algorithm

In [9]:
# 以下，引っ張ってきたソースコードから．
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
from common.functions import *
from common.gradient import numerical_gradient


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 重みの初期化
        # params →　ニューラルネットのパラメータを保持する辞書型変数
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
    
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
        
    # x:入力データ, t:教師データ
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x:入力データ, t:教師データ
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        # それぞれの勾配を保存する．
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        
        batch_num = x.shape[0]
        
        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        # backward
        yt = y - t
        dy = yt / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)

        return grads

In [2]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)

In [5]:
net.params["W1"].shape

(784, 100)

In [6]:
net.params["b1"].shape

(100,)

In [9]:
net.params["W2"].shape

(100, 10)

In [8]:
net.params["b2"].shape

(10,)

要するに，入力層784次元，隠れ層が100次元の2層のニューラルネットワークです．はい．

In [10]:
x = np.random.rand(100, 784)

In [11]:
y = net.predict(x)

In [12]:
y # 100枚の画像（今回はランダムな値）に対してそれぞれ予測したスコアがこちら（もちろん結果も適当）

array([[ 0.09374681,  0.09498098,  0.10096334,  0.1074735 ,  0.09951024,
         0.10187313,  0.1001494 ,  0.10387436,  0.09483882,  0.10258942],
       [ 0.09408458,  0.09529654,  0.10097381,  0.10807615,  0.09920759,
         0.10185238,  0.09984624,  0.10391709,  0.09475228,  0.10199332],
       [ 0.09419853,  0.09543666,  0.10131845,  0.10799299,  0.09901134,
         0.10152247,  0.10003791,  0.10360197,  0.09466218,  0.1022175 ],
       [ 0.09380152,  0.09551234,  0.10109559,  0.1077896 ,  0.09882755,
         0.10191297,  0.0993383 ,  0.1042153 ,  0.09509318,  0.10241365],
       [ 0.09400441,  0.09509371,  0.10096684,  0.10780344,  0.09941714,
         0.10207561,  0.09961887,  0.10413071,  0.09444875,  0.10244053],
       [ 0.09362604,  0.09534267,  0.10104098,  0.10807656,  0.09932347,
         0.10189097,  0.10002753,  0.10386696,  0.09449205,  0.10231278],
       [ 0.09402748,  0.09509935,  0.10116627,  0.10782789,  0.09926916,
         0.10186717,  0.09932044,  0.10412801

In [15]:
x = np.random.rand(100, 784)
t = np.random.rand(100, 10)

In [16]:
grads = net.numerical_gradient(x, t)

In [10]:
from dataset.mnist import load_mnist

In [11]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True)

In [12]:
train_loss_list = []

In [13]:
# ハイパーパラメータ
iter_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

In [14]:
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

In [15]:
for i in range(iter_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    grad = network.gradient(x_batch, t_batch)
    
    for key in ("W1", "b1", "W2", "b2"):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

ValueError: operands could not be broadcast together with shapes (100,10) (100,) 

なんかコケる……．